In [1]:
import requests, bs4, pandas as pd, numpy as np
import mysql.connector, os
from bs4 import BeautifulSoup
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}

# Define fetching function

In [2]:
def soup_(url):
    page = requests.get(url, headers = header)
    soup = BeautifulSoup(page.content,'html.parser')
    soup.prettify()
    return soup

# Fetching States

In [3]:
soup = soup_("https://www.careinsurance.com/health-plan-network-hospitals.html")
items = soup.select('option[value]')
state_values = [item.get('value') for item in items]
state_values = list(map(int, state_values[2:35]))
statelist = soup.find(id = 'searchState').get_text('temp').strip().split('temp')

# Fetching  Cities

In [4]:
cities, citylist = {},[]
for num in state_values:
    soup = soup_("https://www.careinsurance.com/health-plan-network-hospitals.html?state="+str(num))
    items = soup.select('option[value]')
    city_val = [item.get('value') for item in items]
    city_val = list(map(int, city_val[35:]))
    city = soup.find(id = "city").get_text('temp').strip().split('temp')
    citylist.append(city)
    cities[num] = city_val

In [5]:
city_list = []
for i, j in zip(cities.values(), citylist):
    city_list.extend([[x, y] for x,y in zip(i,j)])
city_list = pd.DataFrame(city_list, columns = ['Number', 'Name'])
city_list.set_index("Name", inplace = True)

# Fetching hospitals

In [6]:
hospitals, hosp_list = {},[]
for state in cities:
    print(state)
    for city in cities[state]:
        print("->" + str(city))
        soup = soup_("https://www.careinsurance.com/health-plan-network-hospitals.html?state="+str(state)+"&city="+str(city)+"&search_word=&locator_name=HNL&button=Submit")
        pages = int(soup.find('span', class_='fr').get_text().strip().split(' ')[3])
        var = []
        for page in range(pages):
            soup = soup_("https://www.careinsurance.com/health-plan-network-hospitals.html?state="+str(state)+"&city="+str(city)+"&search_word=&page="+str(page))
            hosps = soup.findAll('code')
            hosps = list(map(lambda s: str(s).rstrip("</code>").lstrip("<code>"), hosps))
            #for itera in range(len(hosps)):
            print(hosps)
                #hosps[itera] = str(hosps[itera]).rstrip("</code>").lstrip("<code>")
            var.extend(hosps)
        hosp_list.append(var)
        hospitals[city] = var 

13
->870
['Srinidhi Hospitals(Knf Road)']
->154
['Aasha Hospitals(Court Road)', 'Dr. YSR Memorial Hospitals(Sai Nagar)', 'Sreenivasa Multispeciality Hospital(Subash Road)', 'Vasan Eye Care Hospital(Anantapur)', 'Divyasree Hospitals(Adimurthy Nagar)', 'Snehalatha Hospitals(Anantapur)', 'Dr. Akbar Eye Hospital (Sai Nagar)', 'Kraanthi Speciality Hospital Pvt Ltd(Vamachandra)', 'Saveera Hospital Pvt Ltd(Srinagar Colony Extension)', 'S R Multi Speciality Hospital(Sangamesh Nagar)']
->445
['LMR Multispeciality Hospital(Behind R.T.C. Bus Stand)', 'Sree Amaravathi Hospital(Aravind Nagar)']
->209
['Narayana Reddy Eye Hospital(Anaparthi)']
->1236
['PADMAVATHI HOSPITAL(SIDDAVATAM ROAD)']
->566
['Jnanananda Retina Foundation Pvt. Ltd.(Bhimavaram)', 'Varma Hospitals(Suryanarayanapuram)', 'Imperial Hospital(Bhimavaram)', 'Bhimavaram Hospital Ltd(Chinamiram)', 'Kasi Eye Hospital(Sri Rampuram)']
->224
['Pragathi Nursing Home(Chilakaluripet)']
->529
['Ravi Nursing Home(Chirala)']
->502
['RVS Hospitals(

['Rich Hospitals(Kasthuridevi Nagar)', 'Sri Vaishnavi Nursing Home(Barracks Centre)', 'Anjali Nursing Home(Ramesh Reddy Nagar)', 'Lotus Hospital(Pogathota)', 'RSR Hospital(Pogathota)', 'Apollo Speciality Hospital(Pinakini Nagar)', 'Kidz Rainbow Children &amp; Baby Hospital(Nellore)', 'Venkataramana Hospital(Gandhi Nagar)', "St. Joseph's Hospital(Santhapet)", 'Sri Ramachandra Nursing Home(Pogathota)']
->173
['Manasa Mother &amp; Child Hospital(ongole)', 'Prakasam Super Speciality Hospital(Beside Rtc Bus Stand)', 'Venkata Ramana Nursing Home Pvt Ltd(opp Apsrtc Bus Depot)', 'Ravi Mother and Child Hospital(Ongole)', 'Modern Eye Hospital And Research Centre(Opp Bethune Nursing Home, Prakasam)', 'Janatha Institute Of Medical Sciences(Court Centre)', 'Vasudhakalyan multi Speciality Hospital(Beside Manasa Hospital sundaraiah Bhavan Road)', 'Royal Multispeciality Hospital(Prakasam)', 'Suraksha Hospital(Beside Jayallukkas Kurnool Road)', 'Vaatsalya Hospital(Opposite Pragathi Bhavan South Bypass)

['Padmaja Hospital(Gajuwaka)', 'Durga Hospitals(Kancharapalem)', "Dr Agarwal's Eye Hospital Ltd.(Vizag)", 'SRI KRISHNA ORTHOPEDIC HOSPITAl(Chinagantyada)', 'Krishna Maternity And Nursing Home(Visakhapatnam)', 'Bhanu Eye Care Centre(Chingantyada)', 'Sunrise Childrens Hospital(Latha Hospital Bldg, Old Gajuwaka)', 'Lotus Hospital For Women And Children(Siripuram Behind Gotisons)', 'Pinnacle Hospitals India Pvt Ltd(Chinnagadilli)', 'Star Pinnacle Health Centre Pvt Ltd(Arilova)']
->170
['Andhra Hospitals(Near Simhachalam Meda)', 'Venkata Rama Hospital(Vizianagaram)', 'Jyothi Nursing Home(A.G. Road)', 'Venkatapadma Hospital(Vizianagaram)', "Care 'N' Cure Children's Hospital(Vizianagaram)", 'Vijaya Hospital(Vizianagaram)', 'Swamy Eye Hospital(Vizianagaram)', 'Maruthi Hospital(Vizianagaram)', 'Queen S Nri Hospital Association With Vijayanagar Hospital Pvt Ltd(Sri Nivas Nagar)', 'A J Global Health Care Hospital(100 Feet Ring Road)']
['Sri Sai Superspeciality Hospital(Balaji Nagar)', 'Sri Sai PV

['New Apollo Hospital(Only For Gramin Care)(Ekma Saran)']
->981
['Mata Mariam Nursing Home(Only For Gramin Care)(Forbesganj Araria)', 'Lions Netralaya(Only For Gramin Care)(Ward no 19)']
->617
['Abhay Institute Of Medical Sciences Pvt Ltd(Gaya)', 'B D Hospital(Only For Gramin Care)(Manpur)', 'Harihar Global Hospital And Research Centre Pvt Ltd(Road No 09)', 'Maa Gayatry Memorial Hospital Pvt Limited(Only For Gramin Care)(Chaurhar More)', 'Buddha Nursing Home(Only For Gramin Care)(Godam Rod)', 'Prakash Multispeciality Hospital(Gewalvigha)', 'Chandra Nursing Home(Only For Gramin Care)(Near Dandibag More)', 'New Sree Ramakrishna Multispeciality Hospital(Only For Gramin Care)(Gaya)']
->747
['Bhagwati Nursing Home(Only For Gramin Care)(Pholparas)']
->889
['Harsan Hospital And Trauma Center A Unit Of Aryan Health Services Pvt Ltd(Only For Gramin Care)(Gopalganj)', 'Gopalganj Eye Hospital(Only For Gramin Care)(In Front Of Gopalganj Railway Station)', 'Shristi Saumya Hospital(Only For Gramin C

['Swaraj Hospital(Ward No-35)', 'Surya Hospital(Only For Gramin Care)(Ward No 8)', 'Nayan Jyoti Netra Chikitsa Kendra(Only For Gramin Care)(Ward No 5)', 'Ganga Hospital(Only For Gramin Care)(Refugee Colony,Ward-5)']
->814
['Adarsh Hospital(Only For Gramin Care)(Ward No 09)', 'Vishnu Sewa Sadan Hospital(Only For Gramin Care)(Adarsh Nagar)', 'Samastipur Eye Hospital(Only For Gramin Care)(Mohanpur Road)', 'Jeevan Sahara Hospital Pvt Ltd(Only For Gramin Care)(Tajpur)', 'J S Hospital(Only For Gramin Care)(Samastipur)', 'Muskan Sewa Sadan(Only For Gramin Care)(Samastipur)', 'Shivangi Nursing Home(Only For Gramin Care)(Samastipur)', 'Aditi Eye Hospital(Only For Gramin Care)(Samastipur)', 'Sharma Hospital(Only For Gramin Care)(Rosera Samastipur)', 'Shri Narayana Emergency Hospital(Only For Gramin Care)(Samstipur)']
->698
['Vikas Nursing Home(Only For Gramin Care)(Sasaram)', 'Narayan Medical College And Hospital(Only For Gramin Care)(Jamuhar)', 'Nirwana Netralaya(Only For Gramin Care)(Atnknambh

...

['Aprajita Hospital(Only For Gramin Care)(Janki Puram)', 'Gopi Krishna Laser Stone Center(Only For Gramin Care)(Aliganj)', 'Lucknow Surgical Hospital(Only For Gramin Care)(Lucknow)', 'Vinayak Medicare Hospital(Only For Gramin Care)(Bijnaur)', 'Precision Urology Hospital(Only For Gramin Care)(Terhi Puliya)', 'Sushma Hospital And Research Centre(Indira Nagar)', 'Silver Line Multispeciality Hospital(Only For Gramin Care)(Lucknow)', 'Brajraj Hospital &amp; Superspecility Center(Only For Gramin Care)(Lucknow)', 'Rida Hospital(Only For Gramin Care)(Lucknow)', 'Centre For Neurosciences &amp; Trauma(Only For Gramin Care)(Lucknow)']
['Lucknow Metro Hospital And Trauma Centre(Lucknow)', 'Apex Trauma And Joint Centre(Only For Gramin Care)(Lucknow)', 'Radius Joint Surgery Hospital(Lucknow)', 'Ahuja Hospital(Only For Gramin Care)(Gomti Nagar)', 'Prabhakar Hospital And Maternity Home(Only For Gramin Care)(Sector-Q)', 'Samra Hospital(Near KGMU Petrol Pump)', 'Jeevan Hospital And Trauma Centre(Only Fo

['Tirupati Eye Centre(Sector-33)', 'I-Care Eye Hospital(Sector 26)', 'Apollo Hospital(Sector 26)', 'Fortis Hospital Noida(Sector 62)', 'VRI LEG Clinic(Noida)', 'Metro Hospital &amp; Heart Institute(Sector 12)', 'Metro Hospital &amp; Heart Institute(Sector 11)', 'Kailash Healthcare Ltd(Noida)', 'Eye Health Clinic(Noida)', 'Indo Gulf Hospital(Serctor - 19)']
->936
['Mkn Netra Jyoti Hospital(Only For Gramin Care)(Patel Nagar)', 'Muskan Hospital And Research Center(Only For Gramin Care)(Orai)', 'Kanha Multispeciality Hospital And Research Centre Pvt Ltd(Only For Gramin Care)(Kalpi Road)']
->918
['Ayushman Multi Hospital(Only For Gramin Care)(Padrauna Disst. Kushinagar)']
->756
['Pt Maikoo Lal Virendra Nath Hospital(Only For Gramin Care)(Pilibhit)', 'Kaushalya Devi Eye Institute(Only For Gramin Care)(Near Chhatri Chauraha)']
->1138
['Simhans Hospital(Near Over Bridge)']
->771
['Sawhney Lions Eye Hospital(Only For Gramin Care)(Moradabad Road)']
->1338
['Narayan Hospital &amp; Trauma Centre(B

['Ganga Valley Hospital(Abhisek Nagar)', 'Metro Hospital &amp; Heart Institute(Sidcul)', 'Swami Bhooma Nand Hospital(Jwalapur)', 'City Hospital(Hardwar)', 'Yog Mata Pilot Baba Multispeciality Hospital(Kankhal)', 'New Dev Bhoomi Hospital &amp; MRC(New haridwar Colony)', 'Maxwell Janjeevan Hospital(Kankhal)', 'Surya Deva Multispeciality And Trauma Centre(Only For Gramin Care)(Delhi Haridwar Highway Bahadrabad)', 'Complete Medical Institute(Jwalapur)', 'Jaya Maxwell Hospital(At Malpur Bongla Bahadrabad)']
->319
['Shree Krishna Hospital(Kashipur )', 'Sanjeevani Multispeciality Hospital(Kashipur Udham Singh Nagar)', 'Sahota Super Speciality Hospital and Neuro Trauma Centre(U S Nagar)', 'Ayushman Multi Speciality Hospital(Moradabad Road)']
->1286
['Sparsh Hospital And Research Center(Khatima)', 'Swastik Hospital Pvt Ltd(Only For Gramin Care)(U S Nagar)']
->462
['Mattrix Hospital And Accident Care(Haldwani)']
->908
['Brijesh Hospital(Ramnagar)']
->829
['Bhardwaj Hospital And Maternity Hospita

['Disha Eye Hospital Hooghly Pvt Ltd(Sheoraphuly)', 'Paramount Health Care(Hooghly)', 'Kanak Nursing Home(Bandel)', 'Re-Life(Uttarpara)', 'Netra Kiran Eye Foundation(Srirampore)', 'Phoenix Hospital(Dankuni)', 'New United Nursing Home(Chandannagar)', 'Arunodaya Healthcare Pvt Ltd(Only For Gramin Care)(Hooghly)', 'Lions Clubs Dhaniakhali J M C H C Hospital(Only For Gramin Care)(Dhaniakhali)', 'Anandmay Seva Niketan(Only For Gramin Care)(Khjurdaha Hooghly)']
->157
['Disha Eye Hospital Hooghly Pvt Ltd(Mourigram)', 'Shree Jain Hospital &amp; Research Centre(Shibpur)', 'Micropoint Nursing Home(Belur)', 'Kalpana Women and Child Care(Howrah)', 'Mcs Diagnostic And Hospital Pvt Ltd(Uluberia)', 'Lords Health Care(Shibpur)', 'Life Care Nursing Home(Bally)', 'Bankra Central Nursing Home(Only For Gramin Care)(P.S.Domjur, Bankra)', 'ILS Hospitals(Oppo site Golabari)', 'Susrut Eye Foundation And Research Centre(Howrah)']
['Sanjiban Hospital(Howrah)', 'Singh Medicare Institute Pvt Ltd(Howrah)', 'Netra 

['Monoranjan Seva Sadan(Only For Gramin Care)(Hoogly)', 'Aesthetic Nursing Home(Only For Gramin Care)(Mogra)']
->317
['Kamada Kinkar Memorial ISAP Glocal Hospital(Murshidabad)', 'Parkland Maternity and Nursing Home(Only For Gramin Care)(Murshidabad)', 'Sathi Nursing Home(Only For Gramin Care)(Raghunathgan)', 'Jeevan Deep Nursing Home And Diagnostic Centre(Only For Gramin Care)(Near Hazi A K Khan Collage)', 'Al Ameen Nursing Home(Only For Gramin Care)(Lalbagh Murshidabad)', 'Well Bred Nursing Home(Only For Gramin Care)(Murshidabad)']
->1082
['Plassey Nabab Nursing Home(Only For Gramin Care)(Ps Kaliganj)', 'Seva Sadan Nursing Home(Only For Gramin Care)(Kaliganj)']
->137
['Greenview Clinic &amp; Nursing Home(R.B.C Road)']
->406
['Anandalok Hospital(Salt Lake)']
->1060
['Sri Durga Maternity and Nursing Home(Baragarh, Debra Bazar)', 'Annapurna Nursing Home(Only For Gramin Care)(Paschim Medinipur)']
->1083
['Plassey Modern Nursing Home(Plassey)']
->1308
['Eveland Nursing Home(Only For Gramin

In [7]:
Data = []
for ST in range(len(statelist)):
    for CT in citylist[ST]:
        for H in hospitals[city_list.loc[CT][0]]:
            Data.append([statelist[ST], CT, H])
        
df = pd.DataFrame(Data, columns=['State', 'City', 'Hospital'])
df

,State,City,Hospital
0,Andhra Pradesh,Amalapuram,Srinidhi Hospitals(Knf Road)
1,Andhra Pradesh,Anantapur,Aasha Hospitals(Court Road)
2,Andhra Pradesh,Anantapur,Dr. YSR Memorial Hospitals(Sai Nagar)
3,Andhra Pradesh,Anantapur,Sreenivasa Multispeciality Hospital(Subash Road)
4,Andhra Pradesh,Anantapur,Vasan Eye Care Hospital(Anantapur)
...,...,...,...
9792,West Bengal,Uluberia,Ideal Clinic And Hospital Pvt Ltd(Uluberia)
9793,West Bengal,Uttarpara,We Care Nursing Home(Uttarpara)
9794,West Bengal,Uttarpara,Nilima Matri Sadan(Uttarpara)
9795,West Bengal,Uttarpara,Sunayan(Sarani)


In [8]:
df.isnull().sum()

State       0
City        0
Hospital    0
dtype: int64

# Using MySql

In [9]:
db_list = []
for itera in range(len(df)):
    db_list.append((df.loc[itera][0], df.loc[itera][1], df.loc[itera][2]))

In [10]:
connection = mysql.connector.connect(
    host = 'localhost',
    user = 'root',    #DB_USER
    passwd = 'adminjain@968',    #DB_PASS
    database = 'hospital_data'
)

In [11]:
mycursor = connection.cursor()
mycursor.execute("DROP TABLE IF EXISTS hospital")
mycursor.execute("CREATE TABLE hospital( State VARCHAR(255), City VARCHAR(255), Hospital VARCHAR(255))")

In [12]:
mycursor.executemany("INSERT INTO hospital(State, City, Hospital) VALUES(%s, %s, %s);",db_list)
connection.commit()

In [13]:
mycursor.execute("SELECT * FROM hospital_data.hospital;")
res = mycursor.fetchall()
res

[('Andhra Pradesh', 'Amalapuram', 'Srinidhi Hospitals(Knf Road)'),
 ('Andhra Pradesh', 'Anantapur', 'Aasha Hospitals(Court Road)'),
 ('Andhra Pradesh', 'Anantapur', 'Dr. YSR Memorial Hospitals(Sai Nagar)'),
 ('Andhra Pradesh',
  'Anantapur',
  'Sreenivasa Multispeciality Hospital(Subash Road)'),
 ('Andhra Pradesh', 'Anantapur', 'Vasan Eye Care Hospital(Anantapur)'),
 ('Andhra Pradesh', 'Anantapur', 'Divyasree Hospitals(Adimurthy Nagar)'),
 ('Andhra Pradesh', 'Anantapur', 'Snehalatha Hospitals(Anantapur)'),
 ('Andhra Pradesh', 'Anantapur', 'Dr. Akbar Eye Hospital (Sai Nagar)'),
 ('Andhra Pradesh',
  'Anantapur',
  'Kraanthi Speciality Hospital Pvt Ltd(Vamachandra)'),
 ('Andhra Pradesh',
  'Anantapur',
  'Saveera Hospital Pvt Ltd(Srinagar Colony Extension)'),
 ('Andhra Pradesh',
  'Anantapur',
  'S R Multi Speciality Hospital(Sangamesh Nagar)'),
 ('Andhra Pradesh',
  'Anantapuramu',
  'LMR Multispeciality Hospital(Behind R.T.C. Bus Stand)'),
 ('Andhra Pradesh', 'Anantapuramu', 'Sree Amar

In [14]:
connection.close()

# Using SQLite3

In [15]:
import sqlite3
conn = sqlite3.connect('Hospital_Data.db')
cur = conn.cursor()
cur.execute('''DROP TABLE IF EXISTS hospital''')
cur.execute('''CREATE TABLE hospital( State TEXT, City TEXT, Hospital TEXT)''')
cur.executemany('''INSERT INTO hospital VALUES(?, ?, ?)''', db_list)
conn.commit()
print("Data is fetched!")
cur.execute('''SELECT * FROM hospital''')
results = cur.fetchall()
conn.close()

Data is fetched!


In [16]:
results

[('Andhra Pradesh', 'Amalapuram', 'Srinidhi Hospitals(Knf Road)'),
 ('Andhra Pradesh', 'Anantapur', 'Aasha Hospitals(Court Road)'),
 ('Andhra Pradesh', 'Anantapur', 'Dr. YSR Memorial Hospitals(Sai Nagar)'),
 ('Andhra Pradesh',
  'Anantapur',
  'Sreenivasa Multispeciality Hospital(Subash Road)'),
 ('Andhra Pradesh', 'Anantapur', 'Vasan Eye Care Hospital(Anantapur)'),
 ('Andhra Pradesh', 'Anantapur', 'Divyasree Hospitals(Adimurthy Nagar)'),
 ('Andhra Pradesh', 'Anantapur', 'Snehalatha Hospitals(Anantapur)'),
 ('Andhra Pradesh', 'Anantapur', 'Dr. Akbar Eye Hospital (Sai Nagar)'),
 ('Andhra Pradesh',
  'Anantapur',
  'Kraanthi Speciality Hospital Pvt Ltd(Vamachandra)'),
 ('Andhra Pradesh',
  'Anantapur',
  'Saveera Hospital Pvt Ltd(Srinagar Colony Extension)'),
 ('Andhra Pradesh',
  'Anantapur',
  'S R Multi Speciality Hospital(Sangamesh Nagar)'),
 ('Andhra Pradesh',
  'Anantapuramu',
  'LMR Multispeciality Hospital(Behind R.T.C. Bus Stand)'),
 ('Andhra Pradesh', 'Anantapuramu', 'Sree Amar

# Save Data to CSV file

In [17]:
df.to_csv('cashless_CAREINSURANCE.csv', index=False)